<a href="https://colab.research.google.com/github/PhilippMatthes/diplom/blob/master/src/shl-deep-learning-timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using a deep CNN to directly classify SHL timeseries data

In [1]:
# Get needed auxiliary files for colab
!git clone https://github.com/philippmatthes/diplom
%cd /content/diplom/src
!mkdir shl-dataset
!wget -nc -O shl-dataset/challenge-2019-train_torso.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_torso.zip
!wget -nc -O shl-dataset/challenge-2019-train_bag.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_bag.zip
!wget -nc -O shl-dataset/challenge-2019-train_hips.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_hips.zip
!wget -nc -O shl-dataset/challenge-2020-train_hand.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2020/challenge-2020-train_hand.zip

fatal: destination path 'diplom' already exists and is not an empty directory.
/content/diplom/src
mkdir: cannot create directory ‘shl-dataset’: File exists
--2021-08-05 08:12:07--  http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_torso.zip
Resolving www.shl-dataset.org (www.shl-dataset.org)... 37.187.125.22
Connecting to www.shl-dataset.org (www.shl-dataset.org)|37.187.125.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5852446972 (5.5G) [application/zip]
Saving to: ‘shl-dataset/challenge-2019-train_torso.zip’

shl-dataset/challen 100%[===================>]   5.45G  3.39MB/s    in 20m 30s 

2021-08-05 08:32:38 (4.54 MB/s) - ‘shl-dataset/challenge-2019-train_torso.zip’ saved [5852446972/5852446972]

--2021-08-05 08:32:38--  http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_bag.zip
Resolving www.shl-dataset.org (www.shl-dataset.org)... 37.187.125.22
Connecting to www.shl-dataset.org (www.shl

In [2]:
from pathlib import Path

MODEL_DIR = Path('models/shl-model-05-08-2021.h5')

In [3]:
# Define all datasets to train our model on

DATASET_DIRS = [
    Path('shl-dataset/challenge-2019-train_torso.zip'),
    Path('shl-dataset/challenge-2019-train_bag.zip'),
    Path('shl-dataset/challenge-2019-train_hips.zip'),
    Path('shl-dataset/challenge-2020-train_hand.zip'),
]

In [4]:
# Create scalers that we will fit on our data

from sklearn.preprocessing import PowerTransformer

acc_scaler = PowerTransformer(method='box-cox')
mag_scaler = PowerTransformer(method='box-cox')
gyr_scaler = PowerTransformer(method='box-cox')

In [5]:
from tensorflow import keras

# Check that we can use our GPU, to not wait forever during training
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10601391199689316843, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 16183459840
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12383643619371656949
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [6]:
# Create our model

from tools.architectures import make_resnet
from tools.dataset import shl_dataset_label_order

if MODEL_DIR.is_file():
    # Load model for transfer learning
    model = keras.models.load_model(MODEL_DIR)
else:
    # Create model
    model = make_resnet(
        input_shape=[500, 3], # Magnitudes of ACC, MAG and Gyr, 500 data points each 
        output_classes=len(shl_dataset_label_order)
    )

In [ ]:
import numpy as np

from tools.dataset import load_zipped_shl_dataset
from tools.export import export_power_transformer

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm

for dataset_dir in DATASET_DIRS:
    # Load dataset from zip file into temporary directory
    dataset = load_zipped_shl_dataset(dataset_dir, tqdm=tqdm)

    # Fit and export scalers
    acc_mag_scaled = acc_scaler.fit_transform(dataset.acc_mag)
    mag_mag_scaled = mag_scaler.fit_transform(dataset.mag_mag)
    gyr_mag_scaled = gyr_scaler.fit_transform(dataset.gyr_mag)
    export_power_transformer(acc_scaler, 'models/acc-transformer.json')
    export_power_transformer(mag_scaler, 'models/mag-transformer.json')
    export_power_transformer(gyr_scaler, 'models/gyr-transformer.json')

    # Prepare our training data
    X = np.stack([
        acc_mag_scaled,
        mag_mag_scaled, 
        gyr_mag_scaled,
    ], axis=2)
    y = np.eye(len(shl_dataset_label_order))[dataset.labels] # One Hot Encoding

    # Save some memory
    del dataset

    # Compute class weights for unbiased training
    y_integers = np.argmax(y, axis=1)
    class_weights = compute_class_weight('balanced', classes=np.unique(y_integers), y=y_integers)
    class_weights = dict(zip(np.unique(y_integers), class_weights)) # Keras adaption

    # Train model
    callbacks = [
        keras.callbacks.CSVLogger(f'{MODEL_DIR}.log', append=MODEL_DIR.is_file()),
        keras.callbacks.ModelCheckpoint(
            str(MODEL_DIR), save_best_only=True, monitor='val_loss'
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001
        ),
        
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1),
    ]
    history = model.fit(
        X, 
        y,
        epochs=10, 
        batch_size=32,
        class_weight=class_weights, 
        callbacks=callbacks,
        validation_split=0.1,
        verbose=1,
    )
    

Extracting shl-dataset/challenge-2019-train_torso.zip:  27%|██▋       | 6/22 [00:44<02:05,  7.85s/it]